## EXPLORACIÓN Y LIMPIEZA INICIAL DE LOS DATOS

En este archivo nos dedicaremos a explorar, limpiar y transformar nuestros datos para adecuarlos y orientarlos a sacar el máximo rendimiento en nuestros modelos de Machine Learning.

In [3]:
# Importación de recursos

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import pylab as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Cargamos y comprobamos la existencia de valores nulos.

In [4]:
salaries = pd.read_csv('../data/salaries_data.csv')

salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [5]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           500 non-null    int64 
 1   experience_level    500 non-null    object
 2   employment_type     500 non-null    object
 3   job_title           500 non-null    object
 4   salary              500 non-null    int64 
 5   salary_currency     500 non-null    object
 6   salary_in_usd       500 non-null    int64 
 7   employee_residence  500 non-null    object
 8   remote_ratio        500 non-null    int64 
 9   company_location    500 non-null    object
 10  company_size        500 non-null    object
dtypes: int64(4), object(7)
memory usage: 43.1+ KB


In [ ]:
# Comprobamos que la mayoría de columnas son categóricas, las cuales habrá que transformar para aplicar modelos de ML.

In [ ]:
# Las siguientes celdas están destinadas al análisis de nuestros datos orientado a detectar diferencias significativas
# en los salarios, agrupando los datos de diversas maneras, para poder codificarlos de manera adecuada.

In [5]:
salaries.salary_currency.unique()

array(['USD', 'EUR', 'INR', 'GBP', 'JPY', 'CAD', 'MXN', 'CHF', 'CLP',
       'TRY', 'AUD', 'BRL', 'HUF', 'PLN', 'SGD'], dtype=object)

In [ ]:
# Agrupo las monedas por continentes

In [6]:
continent = []

for e in salaries.salary_currency:
    if e == 'USD':
        continent.append('AME')
    if e == 'EUR':
        continent.append('EUR')
    if e == 'INR':
        continent.append('ASI')
    if e == 'GBP':
        continent.append('EUR')
    if e == 'JPY':
        continent.append('ASI')
    if e == 'CAD':
        continent.append('AME')
    if e == 'MXN':
        continent.append('LTM')
    if e == 'CHF':
        continent.append('EUR')
    if e == 'CLP':
        continent.append('LTM')
    if e == 'TRY':
        continent.append('ASI')
    if e == 'AUD':
        continent.append('AUS')
    if e == 'BRL':
        continent.append('LTM')
    if e == 'HUF':
        continent.append('EUR')
    if e == 'PLN':
        continent.append('EUR')
    if e == 'SGD':
        continent.append('ASI')

In [7]:
salaries['Continent'] = continent

In [ ]:
# Compruebo que hay una diferencia significativa en los sueldos dependiendo de la parte del mundo de donde provengan.

In [8]:
salaries.groupby('Continent')['salary_in_usd'].mean().sort_values(ascending = False)

Continent
AME    135574.089855
AUS     87064.000000
EUR     70662.116667
ASI     33655.366667
LTM     20601.333333
Name: salary_in_usd, dtype: float64

In [ ]:
# De manera similiar nos encontramos con diferencias según el nivel de experiencia.

In [9]:
salaries.groupby('experience_level')['salary_in_usd'].mean().sort_values(ascending = False)

experience_level
EX    196078.850000
SE    136341.794239
MI     90384.610465
EN     59989.569231
Name: salary_in_usd, dtype: float64

In [ ]:
# Lo mismo ocurre con el tipo de empleo.

In [10]:
salaries.groupby('employment_type')['salary_in_usd'].mean().sort_values(ascending = False)

employment_type
CT    222750.000000
FT    113914.630165
FL     48000.000000
PT     35053.375000
Name: salary_in_usd, dtype: float64

In [ ]:
# El análisis previo nos sirve para etiquetar estas columnas categóricas de manera que nuestros modelos puedan 
# interpretar estos datos de manera correcta. Este proceso genera un sesgo, pero no es un sesgo infundado, ya que 
# hemos comprobado que hay grandes diferencias.

In [12]:
dictio_typ = {'PT':0, 'FL':1, 'FT':2, 'CT':3}
salaries.employment_type = salaries.employment_type.apply(lambda x: dictio_typ[x])

In [13]:
dictio_curr = {'USD': 4, 'EUR': 2, 'INR': 1, 'GBP': 2, 'JPY': 2, 'CAD': 4, 'MXN':0, 'CHF':2, 'CLP':0, 'TRY':1, 'AUD':3, 'BRL':0, 'HUF':2, 'PLN':2, 'SGD':1}

salaries.salary_currency = salaries.salary_currency.apply(lambda x: dictio_curr[x])

salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Continent
0,2022,SE,2,Data Engineer,140250,4,140250,US,100,US,M,AME
1,2022,SE,2,Data Engineer,135000,4,135000,US,100,US,M,AME
2,2021,MI,2,BI Data Analyst,100000,4,100000,US,100,US,M,AME
3,2021,MI,3,ML Engineer,270000,4,270000,US,100,US,L,AME
4,2021,MI,2,Data Engineer,22000,2,26005,RO,0,US,L,EUR


In [14]:
dictio = {'EN': 0, 'MI': 1, 'SE': 2, 'EX':3}


salaries.experience_level = salaries.experience_level.apply(lambda x: dictio[x])

salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Continent
0,2022,2,2,Data Engineer,140250,4,140250,US,100,US,M,AME
1,2022,2,2,Data Engineer,135000,4,135000,US,100,US,M,AME
2,2021,1,2,BI Data Analyst,100000,4,100000,US,100,US,M,AME
3,2021,1,3,ML Engineer,270000,4,270000,US,100,US,L,AME
4,2021,1,2,Data Engineer,22000,2,26005,RO,0,US,L,EUR


In [ ]:
# Transformamos la columna 'job_title' mediante la siguiente función para realizar un proceso similar a los anteriores.

In [1]:
def job_title(data):
    if 'Head' in data:
        return 'Data_Head'
    elif 'Director' in data:
        return 'Data_Director'
    elif 'Lead' in data:
        return 'Data_Lead'
    elif 'Principal' in data:
        return 'Data_Princial'
    else:
        return 'Data_Other'

In [15]:
salaries.job_title = salaries.job_title.apply(job_title)

salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,Continent
0,2022,2,2,Data_Other,140250,4,140250,US,100,US,M,AME
1,2022,2,2,Data_Other,135000,4,135000,US,100,US,M,AME
2,2021,1,2,Data_Other,100000,4,100000,US,100,US,M,AME
3,2021,1,3,Data_Other,270000,4,270000,US,100,US,L,AME
4,2021,1,2,Data_Other,22000,2,26005,RO,0,US,L,EUR


In [16]:
salaries.job_title.unique()

array(['Data_Other', 'Data_Princial', 'Data_Lead', 'Data_Director',
       'Data_Head'], dtype=object)

In [ ]:
# Eliminamos algunas columnas que no nos sirven

In [17]:
salaries.drop(['Continent'], axis = 1, inplace = True)

In [18]:
salaries.drop('salary', axis = 1, inplace = True)

In [19]:
salaries.company_location.unique()

array(['US', 'FR', 'GR', 'LU', 'SI', 'DE', 'IN', 'GB', 'PK', 'MD', 'JP',
       'CA', 'AS', 'IE', 'AE', 'MX', 'VN', 'BE', 'KE', 'ES', 'CH', 'CL',
       'CN', 'DK', 'TR', 'NZ', 'PL', 'UA', 'AU', 'NG', 'EE', 'CZ', 'AT',
       'BR', 'DZ', 'IR', 'NL', 'HU', 'PT', 'HN', 'MT', 'RO', 'SG'],
      dtype=object)

In [20]:
salaries.employee_residence.value_counts()[:10]

US    278
GB     38
IN     25
CA     24
DE     19
FR     14
ES     13
GR     11
PK      6
PT      6
Name: employee_residence, dtype: int64

In [ ]:
# Determino que 'employee_residence' nos nos aporta demasiado valor ya que son datos que en su mayoría son redundantes
# con respecto a 'company_location'. Opto por eliminarla.

In [21]:
salaries.drop('employee_residence', axis = 1, inplace = True)

In [ ]:
# Proceso de etiquetación similaar a los anteriores, en esta ocasión con la localización de la empresa.
# Cabe recalcar que las etiquetas se ponen en función de la media de salario para cada grupo. En este caso, según 
# la media de salarios por continentes.

In [23]:
dictio_comp_loc = {'US':'AME', 'GB':'EUR', 'CA':'AME', 'DE':'EUR', 'IN':'ASI', 'FR':'EUR', 'ES':'EUR', 'GR':'EUR', 'JP':'ASI', 'NL':'EUR', 'PT':'EUR', 'LU':'EUR', 'AE':'ASI', 'AT':'EUR', 'PL':'EUR', 'PK':'ASI', 'TR':'ASI', 'CH':'EUR', 'AU':'AUS', 'BR':'LTM', 'NG':'AFR', 'SI':'EUR', 'BE':'EUR', 'MT':'EUR', 'IR':'EUR', 'DZ':'AFR', 'HU':'EUR', 'CZ':'EUR', 'HN':'LTM', 'RO':'EUR', 'CL':'LTM', 'EE':'EUR', 'UA':'EUR', 'NZ':'AUS', 'DK':'EUR', 'CN':'ASI', 'KE':'AFR', 'VN':'ASI', 'MX':'LTM', 'IE':'EUR', 'AS':'AME', 'MD':'EUR', 'SG':'ASI'}

salaries.company_location = salaries.company_location.apply(lambda x: dictio_comp_loc[x])

In [24]:
salaries.groupby('company_location')['salary_in_usd'].mean().sort_values(ascending=False)

company_location
AME    141093.854037
AUS     99709.333333
EUR     68349.083969
ASI     43941.514286
AFR     42318.000000
LTM     21160.800000
Name: salary_in_usd, dtype: float64

In [25]:
dictio_cont_lab = {'LTM':0, 'AFR':1, 'ASI':2, 'EUR':3, 'AUS':4, 'AME':5}

salaries.company_location = salaries.company_location.apply(lambda x: dictio_cont_lab[x])

In [ ]:
# Aplicamos técnicas de one-hot-encoding sobre las columnas categóricas restantes.

In [26]:
salaries = pd.get_dummies(salaries, columns = ['company_size', 'job_title'], drop_first = True)

In [ ]:
# Definimos nuestra columna objetivo y dividimos el df en función de esta decisión.

In [ ]:
X = salaries.drop('salary_in_usd', axis = 1)
y = salaries.salary_in_usd

In [ ]:
# Aplicamos funciones de normalización a nuestros datos, de diferentes maneras para testear su rendimiento.

In [ ]:
# Normalización 1
# StackedEnsemble_AllModels_1_AutoML_4_20230304_172907	54553.7	2.97611e+09	37721.5	0.555565	2.97611e+09

from sklearn.preprocessing import StandardScaler    

data_num=pd.DataFrame(StandardScaler().fit_transform(salaries[['work_year', 'remote_ratio', 'experience_level', 'employment_type', 'salary_currency', 'company_location']]),
                      
                      columns= ['work_year', 'remote_ratio', 'experience_level', 'employment_type', 'salary_currency', 'company_location'])


data_obj=salaries[['company_size_M', 'company_size_S', 'salary_in_usd']]

data=pd.concat([data_num, data_obj], axis=1)

In [29]:
# Normalización 2
# StackedEnsemble_AllModels_1_AutoML_3_20230304_172513	53959.1	2.91158e+09	37550.4	0.571871	2.91158e+09

from sklearn.preprocessing import StandardScaler    

data_num=pd.DataFrame(StandardScaler().fit_transform(salaries[['work_year', 'remote_ratio']]),
                      
                      columns= ['work_year', 'remote_ratio'])


data_obj=salaries[['job_title_Data_Head', 'job_title_Data_Lead', 'job_title_Data_Other', 'job_title_Data_Princial', 'company_size_M', 'company_size_S', 'salary_in_usd', 'experience_level', 'employment_type', 'salary_currency', 'company_location']]

data=pd.concat([data_num, data_obj], axis=1)

In [30]:
data.head()

,work_year,remote_ratio,job_title_Data_Head,job_title_Data_Lead,job_title_Data_Other,job_title_Data_Princial,company_size_M,company_size_S,salary_in_usd,experience_level,employment_type,salary_currency,company_location
0,0.857874,0.697464,0,0,1,0,1,0,140250,2,2,4,5
1,0.857874,0.697464,0,0,1,0,1,0,135000,2,2,4,5
2,-0.606075,0.697464,0,0,1,0,1,0,100000,1,2,4,5
3,-0.606075,0.697464,0,0,1,0,0,0,270000,1,3,4,5
4,-0.606075,-1.775814,0,0,1,0,0,0,26005,1,2,2,5


In [ ]:
# A partir de este momento, se hacen algunas pruebas con herramientas y técnicas de testeo de modelos sin llegar al 
# resultado esperado. 
# En el resto de archivos de este repositorio se exponen los avances en este proyecto.

In [31]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test  = tts(data, test_size=0.2, random_state=42)

In [32]:
import h2o

from h2o.automl import H2OAutoML

In [33]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 day 0 hours 36 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_Pedro_6usz9y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.406 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [34]:
h2train=h2o.H2OFrame(X_train)
h2test=h2o.H2OFrame(X_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [35]:
h2train.columns

['work_year',
 'remote_ratio',
 'job_title_Data_Head',
 'job_title_Data_Lead',
 'job_title_Data_Other',
 'job_title_Data_Princial',
 'company_size_M',
 'company_size_S',
 'salary_in_usd',
 'experience_level',
 'employment_type',
 'salary_currency',
 'company_location']

In [36]:
X=[c for c in h2train.columns if c!='salary_in_usd']

y='salary_in_usd'

In [43]:
automl=H2OAutoML(max_models=20,
                 seed=42,   # random_state
                 max_runtime_secs=3000,
                 sort_metric='RMSE')

In [44]:
automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),6/20
# GBM base models (used / total),2/7
# XGBoost base models (used / total),1/6
# DeepLearning base models (used / total),3/4
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [45]:

print('[INFO] Leader board:')

leader_board=automl.leaderboard

leader_board.head()

[INFO] Leader board:


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_7_20230304_181307,52919.6,2.80048e+09,36624.4,nan,2.80048e+09
DeepLearning_grid_1_AutoML_7_20230304_181307_model_1,53750.3,2.88909e+09,38384.6,nan,2.88909e+09
DeepLearning_grid_2_AutoML_7_20230304_181307_model_1,53809.4,2.89545e+09,38210.2,nan,2.89545e+09
GBM_4_AutoML_7_20230304_181307,53981.1,2.91396e+09,36846.4,0.552549,2.91396e+09
StackedEnsemble_BestOfFamily_1_AutoML_7_20230304_181307,54119.6,2.92893e+09,37265,0.549316,2.92893e+09
GBM_3_AutoML_7_20230304_181307,54348,2.9537e+09,37024.8,0.553762,2.9537e+09
GBM_2_AutoML_7_20230304_181307,54447.1,2.96449e+09,37306.2,0.549804,2.96449e+09
GBM_grid_1_AutoML_7_20230304_181307_model_2,54478.8,2.96794e+09,36932.5,0.542937,2.96794e+09
DeepLearning_1_AutoML_7_20230304_181307,55015.7,3.02672e+09,37786.7,0.593607,3.02672e+09
XGBoost_1_AutoML_7_20230304_181307,55180.1,3.04484e+09,38224.8,nan,3.04484e+09


In [ ]:
y_pred=automl.leader.predict(h2test)

In [ ]:
y_pred[:10]